# Load Libraries


In [20]:
import requests
from datetime import datetime
import prettytable
from geopy.geocoders import Nominatim

from IPython.core.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))


# Main program

In [24]:
# For more documentation/info api data used in this code refer to this page
#  https://openweathermap.org/api/one-call-api#current


apiKey = "05d26a856cc9acf4026b4d3175cbcde3"
lat = 48.7616888
lon = -91.6271136
part = "minutely,hourly"
units = 'metric'
lang = 'en'

oneCallUrl = f"http://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={part}" \
             f"&appid={apiKey}&lang={lang}&units={units}"
response = requests.get(oneCallUrl)


def alertParser(alerts):
    for alert in alerts:
        alert["startStr"] = datetime.utcfromtimestamp(alert['start']).strftime('%A, %B %d %Y at %H:%M %p UTC')
        alert["endStr"] = datetime.utcfromtimestamp(alert['end']).strftime('%A, %B %d %Y at %H:%M %p UTC')
    return alerts


def currentDataParser(current):
    current["dateStr"] = datetime.utcfromtimestamp(current['dt']).strftime('%A, %B %d %Y at %H:%M %p UTC')
    return current


def getLocationName(latitude, longitude):
    locator = Nominatim(user_agent="myGeocoder")
    coords = str(latitude) + "," + str(longitude)
    locationName = locator.reverse(coords)
    if locationName is not None and ({"city", "address", "country"} <= set(locationName.raw['address'])):
        return f"{locationName.raw['address']['city']}, " \
               f"{locationName.raw['address']['state']}, " \
               f"{locationName.raw['address']['country']}"
    else:
        return locationName.raw['display_name']


def printCurrentWeather(currentLocation, current):
    print(f"********************************** Weather Data for  {currentLocation} **********************************")
    print("\nCurrent Weather")
    print("-------------------")

    print(f"Current Temperature: {current['temp']} C\u00b0")
    print(f"Feels like: {current['feels_like']} C\u00b0")
    print(f"Conditions: {current['weather'][0]['description']} C\u00b0")


def print7DayForecast(daily):
    print("\n7 Day Forecast")
    listOfDays = list(map(lambda x: datetime.utcfromtimestamp(x['dt']).strftime('%a, %B %d %Y '), daily))
    table = prettytable.PrettyTable()
    for i, item in enumerate(daily):
        maxTemp = "Max: " + str(item['temp']['max']) + " C\u00b0"
        minTemp = "Min: " + str(item['temp']['min']) + " C\u00b0"
        table.add_column(listOfDays[i], [maxTemp, minTemp])
    print(table)


def printAlerts(data):
    print(f"\n********************************** Weather Alerts **********************************")
    if 'alerts' not in data:
        print("No current alerts were found for this location ")
    else:
        alertData = alertParser(data['alerts'])
        for alert in alertData:
            sender = alert['sender_name']
            eventName = alert['event']
            startTime = alert['startStr']
            endTime = alert['endStr']
            description = alert['description']

            print(startTime)
            print(f"{eventName.capitalize()} advisory in effect")
            print(f"Active until {endTime}")
            print(f"issued by: {sender}")
            print("----------------------------------------------")
            print("Description")
            print(description)


def main():
    if response.status_code == 200:
        data = response.json()
        currentData = currentDataParser(data['current'])
        dailyTemp = data['daily']
        getLocationFromCoords = getLocationName(lat, lon)

        printCurrentWeather(getLocationFromCoords, currentData)
        print7DayForecast(dailyTemp)
        printAlerts(data)

    else:
        print(
            f"Response returned the following error code: {response.status_code} -- {response.reason}")


if __name__ == "__main__":
    main()


********************************** Weather Data for  Hancock Boulevard, Atikokan, Rainy River District, Northwestern Ontario, Ontario, Canada **********************************

Current Weather
-------------------
Current Temperature: 25.83 C°
Feels like: 25.67 C°
Conditions: scattered clouds C°

7 Day Forecast
+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+
| Thu, September 16 2021  | Fri, September 17 2021  | Sat, September 18 2021  | Sun, September 19 2021  | Mon, September 20 2021  | Tue, September 21 2021  | Wed, September 22 2021  | Thu, September 23 2021  |
+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+
|      Max: 26.17 C°      |      Max: 15.62 C°      |    